In [9]:
import pprint
import numpy as np
import math
np.set_printoptions(precision=3)

In [2]:
# 上位K件
TOP_K = 3

# 近傍アイテム数
K_ITEMS = 3

# しきい値
THETA = 0

In [3]:
Du = np.array([
               [5, 3, +1],
               [6, 2, +1],
               [4, 1, +1],
               [8, 5, -1],
               [2, 4, -1],
               [3, 6, -1],
               [7, 6, -1],
               [4, 2, np.nan],
               [5, 1, np.nan],
               [8, 6, np.nan],
               [3, 4, np.nan],
               [4, 7, np.nan],
               [4, 4, np.nan],
])

In [4]:
I = np.arange(Du.shape[0])
x = Du[:,:-1]
ru = Du[:,-1]

In [5]:
Iu = I[~np.isnan(ru)]
Iup = I[ru==+1]
Iun = I[ru==-1]
Iu_not = np.setdiff1d(I, Iu)

## 01 ユークリッド距離

In [26]:
def dist(xi, xj):
    """
    距離関数：アイテムiの特徴ベクトルxiとアイテムjの特徴ベクトルxjのユークリッド距離を返す。

    Parameters
    ----------
    xi : ndarray
        アイテムiの特徴ベクトル
    xj : ndarray
        アイテムjの特徴ベクトル

    Returns
    -------
    float
        ユークリッド距離
    """
    # Q!
    
    distance = np.sqrt(np.sum((xi - xj) ** 2))
    
    return distance

In [27]:
i = 7
j = 2
print('dist(x{}, x{}) = {:.3f}'.format(i, j, dist(x[i], x[j])))

i = 7
j = 3
print('dist(x{}, x{}) = {:.3f}'.format(i, j, dist(x[i], x[j])))

dist(x7, x2) = 1.000
dist(x7, x3) = 5.000


## 02 アイテム-アイテム距離行列

In [117]:
D = np.array([[dist(Xarray, Yarray) for Xarray in x] for Yarray in x])
print('D = \n{}'.format(D[np.ix_(Iu_not,Iu)]))

D = 
[[1.414 2.    1.    5.    2.828 4.123 5.   ]
 [2.    1.414 1.    5.    4.243 5.385 5.385]
 [4.243 4.472 6.403 1.    6.325 5.    1.   ]
 [2.236 3.606 3.162 5.099 1.    2.    4.472]
 [4.123 5.385 6.    4.472 3.606 1.414 3.162]
 [1.414 2.828 3.    4.123 2.    2.236 3.606]]


## 03 距離の昇順に並べ替えたインデックスの配列

In [125]:
Di = np.array([[dist(Xarray, Yarray) for Xarray in x[~np.isnan(ru)]] for Yarray in x])

Ii = np.argsort(Di)

print('Ii = \n{}'.format(Ii))

Ii = 
[[0 1 2 4 3 5 6]
 [1 0 2 3 6 4 5]
 [2 0 1 4 5 3 6]
 [3 6 0 1 5 2 4]
 [4 5 0 2 1 6 3]
 [5 4 0 6 1 2 3]
 [6 3 0 5 1 4 2]
 [2 0 1 4 5 3 6]
 [2 1 0 4 3 5 6]
 [3 6 0 1 5 4 2]
 [4 5 0 2 1 6 3]
 [5 6 4 0 3 1 2]
 [0 4 5 1 2 6 3]]


## 04 近傍k件のアイテムのインデックス配列

In [128]:
Ii = Ii[:, :TOP_K]

print('Ii = \n{}'.format(Ii))

Ii = 
[[0 1 2]
 [1 0 2]
 [2 0 1]
 [3 6 0]
 [4 5 0]
 [5 4 0]
 [6 3 0]
 [2 0 1]
 [2 1 0]
 [3 6 0]
 [4 5 0]
 [5 6 4]
 [0 4 5]]


## 05 各対象アイテムの近傍アイテム集合

In [146]:
Ii = {i:Ii[i] for i in Iu_not}

print('Ii = ')
pprint.pprint(Ii)

Ii = 
{7: array([2, 0, 1]),
 8: array([2, 1, 0]),
 9: array([3, 6, 0]),
 10: array([4, 5, 0]),
 11: array([5, 6, 4]),
 12: array([0, 4, 5])}


## 嗜好予測（多数決方式）

## 06 近傍アイテム集合のうち「好き」と評価したアイテム集合

In [172]:
i = 7

In [175]:
Ii[i][np.isin(Ii[i], Iup)]

array([2, 0, 1])

## 07 近傍アイテム集合のうち「嫌い」と評価したアイテム集合 

In [176]:
Ii[i][np.isin(Ii[i], Iun)]

array([], dtype=int64)

## 08 多数決方式による予測評価値

In [181]:
(1 * len(Ii[i][np.isin(Ii[i], Iup)]) + -1 * len(Ii[i][np.isin(Ii[i], Iun)])) / (len(Ii[i][np.isin(Ii[i], Iup)]) + len(Ii[i][np.isin(Ii[i], Iun)]))

1.0

In [182]:
def predict1(u, i):
    """
    予測関数（多数決方式）：多数決方式によりユーザuのアイテムiに対する予測評価値を返す。

    Parameters
    ----------
    u : int
        ユーザuのID（ダミー）
    i : int
        アイテムiのID

    Returns
    -------
    float
        予測評価値
    """
    # Q6
    Iip = Ii[i][np.isin(Ii[i], Iup)]
    print('I{}+ = {}'.format(i, Iip))

    # Q7
    Iin = Ii[i][np.isin(Ii[i], Iun)]
    print('I{}- = {}'.format(i, Iin))
    
    # Q08
    rui = (1 * len(Ii[i][np.isin(Ii[i], Iup)]) + -1 * len(Ii[i][np.isin(Ii[i], Iun)])) / (len(Ii[i][np.isin(Ii[i], Iup)]) + len(Ii[i][np.isin(Ii[i], Iun)]))

    return rui

In [183]:
u = 0
i = 7
print('predict1({}, {}) = {:.3f}'.format(u, i, predict1(u, i)))
u = 0
i = 9
print('predict1({}, {}) = {:.3f}'.format(u, i, predict1(u, i)))

I7+ = [2 0 1]
I7- = []
predict1(0, 7) = 1.000
I9+ = [0]
I9- = [3 6]
predict1(0, 9) = -0.333


## 09 平均方式による予測評価値